# Setup

In [7]:
import os

import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import tensorflow_datasets as tfdata

import pennylane as qml
from pennylane import numpy as np

import sklearn
import numpy
import pandas as pd
from sklearn.decomposition import FastICA as fica
from sklearn.decomposition import PCA 
import matplotlib.pyplot as plt
from PIL import Image
import librosa

import pickle

tfk = tf.keras
tfl = tfk.layers
tfd = tfp.distributions
tfpl = tfp.layers

In [8]:
def writeRecords(x, e, y, path='openmic_imagedata.tfrecords'):
    # Write the records to a file.
    with tf.io.TFRecordWriter(path) as file_writer:
        for i in range(len(y)):
            b = x[i].flatten().tolist()
            enc = e[i].tolist()
            #print(b)
            v = float(y[i])
            record_bytes = tf.train.Example(features=tf.train.Features(feature={
                "x": tf.train.Feature(float_list = tf.train.FloatList(value=b)
                    ),
                "e": tf.train.Feature(float_list=tf.train.FloatList(value=enc)),
                "y": tf.train.Feature(float_list=tf.train.FloatList(value=[v])),
            })).SerializeToString()
            file_writer.write(record_bytes)

def decode_fn(record_bytes):
    # Read the data back out.
    return tf.io.parse_single_example(
      # Data
      record_bytes,
      # Schema
      {"x": tf.io.FixedLenFeature((32,32,3), dtype=tf.float32),
       "e": tf.io.FixedLenFeature((12), dtype=tf.float32),
       "y": tf.io.FixedLenFeature((), dtype=tf.float32)}
  )

In [9]:
def create_cabinet_member(kernels=[5,3,3,5], conv_size=6, activation='sigmoid'):
    datum = tfl.Input(shape=(32,32,3),name='x')
#    init = tfk.initializers.Orthogonal()
    activation = activation
    
    c = tfpl.Convolution2DReparameterization(filters=conv_size, kernel_size=kernels[0], padding='valid', data_format='channels_last')(datum)
    x = tfpl.Convolution2DReparameterization(filters=conv_size, kernel_size=kernels[1], padding='valid', data_format='channels_last')(datum)
    x = tfl.Activation(activation)(x)
    x = tfpl.Convolution2DReparameterization(filters=conv_size, kernel_size=kernels[2], padding='valid', data_format='channels_last')(x)
    x = tfl.Add()([x, c])
    x = tfl.Activation(activation)(x)
    x = tfpl.Convolution2DReparameterization(filters=3, kernel_size=kernels[3], padding='valid', data_format='channels_last')(x)
    x = tfl.Activation(activation)(x)
    x = tfl.Flatten()(x)
    x = tfpl.DenseReparameterization(256, activation)(x)
    x = tfpl.DenseReparameterization(10, 'softmax')(x)
    
    return tfk.models.Model(inputs=datum, outputs=x)

In [97]:
# Original Version. Use this for my_model
class QuantumLegislator(tfk.Model):
    def __init__(self, kernels=[5,3,3,5], conv_size=6, num_experts = 4, activation='sigmoid', pres_step_size=.85,batch_size=8):
        super(QuantumLegislator, self).__init__()
        self.num_experts = num_experts
        self.cabinet = [self.create_cabinet_member(kernels=kernels, conv_size=conv_size, activation = activation)] * num_experts
        #self.sample_dev = qml.device('default.qubit', wires=num_experts,shots=1)
        self.state_dev = qml.device('default.qubit', wires=num_experts)
        self.pres_opt = qml.optimize.AdamOptimizer(pres_step_size)
        #self.qvc_weights = np.random.rand(self.num_experts*3, requires_grad=True).reshape(self.num_experts,3)
        self.batch_size= batch_size
        self.pres = self.pres_layer()
        self.accuracy = tf.keras.metrics.CategoricalAccuracy()

    def train_step(self, data):
        x, e, y = data
        #print(type(y))
        #y_gt = np.array([y] * self.num_experts).reshape((self.num_experts,self.batch_size,10)) # num_experts x 10
        #print(y_gt.shape)
        with tf.GradientTape() as tape:
            tom_vetos = self.pres(inputs=e, training=True)[...,1]
            
            y_pred = np.array([cab(x, training = True) for cab in self.cabinet]) # num_experts x batch_size x 10
            cab_loss = [self.compiled_loss(y, cab(x, training = True)) for cab in self.cabinet]

            ## BRYAN LOSS
            tom_sum = tf.reduce_sum(tom_vetos)
            cab_sum = tf.reduce_sum(cab_loss)
            tom_val = tf.cast((tom_vetos - tom_sum), dtype = tf.float32)

            cab_val = tf.reshape(tf.cast((cab_loss-cab_sum), dtype = tf.float32),(self.num_experts))
            expert_losses = tom_val * cab_val
            
            final_loss = tf.reduce_sum(expert_losses)
            weights = self.trainable_variables   # num_experts 
            classic_gradients = tape.gradient(final_loss, weights)
            self.optimizer.apply_gradients(zip(classic_gradients, weights))
            self.accuracy.update_state(y, y_pred)

        return {'Full Loss':final_loss, 'Accuracy': self.accuracy.result()}
    
    def test_step(self, data):
        x, e, y = data
        
        tom_vetos = tf.reshape(self.pres(inputs=e, training=True)[...,1], (self.batch_size * self.num_experts)) # 128,
        vetos = np.array([0 if np.random.rand() > x else 1 for x in tom_vetos]).reshape((1, self.batch_size, self.num_experts )).T # 4X32
        #print(vetos.shape)
        y_pred = np.array([cab(x, training = True) for cab in self.cabinet]) * vetos # num_experts
        y_pred = tf.convert_to_tensor(np.sum(y_pred,axis=0)) # 32 X 10
        #print(y_pred)
        #print(y_pred.shape)
        
        
        cab_loss = [float(self.compiled_loss(y[i], y_pred[i])) for i in range(self.batch_size)]
        self.accuracy.update_state(y, y_pred)

        total_loss = tf.reduce_sum(cab_loss)
        #print("Cab", cab_loss)
        #print("Tot", total_loss.shape)
        return {'Validation Loss':total_loss, 'Validation Accuracy': self.accuracy.result()}

    def create_cabinet_member(self, kernels=[5,3,3,5], conv_size=6, activation='sigmoid'):
        datum = tfl.Input(shape=(32,32,3),name='x')
        #init = tfk.initializers.Orthogonal()
        activation = activation

        c = tfpl.Convolution2DReparameterization(filters=conv_size, kernel_size=kernels[0], padding='valid', data_format='channels_last')(datum)
        x = tfpl.Convolution2DReparameterization(filters=conv_size, kernel_size=kernels[1], padding='valid', data_format='channels_last')(datum)
        x = tfl.Activation(activation)(x)
        x = tfpl.Convolution2DReparameterization(filters=conv_size, kernel_size=kernels[2], padding='valid', data_format='channels_last')(x)
        x = tfl.Add()([x, c])
        x = tfl.Activation(activation)(x)
        x = tfpl.Convolution2DReparameterization(filters=3, kernel_size=kernels[3], padding='valid', data_format='channels_last')(x)
        x = tfl.Activation(activation)(x)
        x = tfl.Flatten()(x)
        x = tfpl.DenseReparameterization(256, activation)(x)
        x = tfpl.DenseReparameterization(10, 'softmax')(x)

        return tfk.models.Model(inputs=datum, outputs=x)
    
    def pres_layer(self):
        #@qml.batch_input(argnum=1)
        @qml.qnode(self.state_dev, diff_method="backprop", interface="tf")
        def president_tom(inputs, weights):
            # x Shape = num_experts * 3
            # Weights Shape = num_experts * 2 * 3
            for i in range(self.num_experts):
                qml.Hadamard(i)
                qml.Rot(inputs[...,3*i],inputs[...,3*i+1],inputs[...,3*i+2],i)
                
                
            qml.CZ((self.num_experts-1, 0)) #entangling ring
            for i in range(1, self.num_experts):
                qml.CZ((i-1, i))
                
            for i in range(self.num_experts):
                qml.Rot(weights[3*i],weights[3*i+1],weights[3*i+2],i)
                #qml.Hadamard(i)
                
            #for i in range(self.num_experts):
                #qml.Rot(weights[i][1][0],weights[i][1][1],weights[i][1][2],i)
            #return [qml.expval(qml.PauliZ(wires=i)) for i in range(self.num_experts)]
            return [qml.probs(x) for x in range(self.num_experts)]
    
        weight_shapes = {"weights":(self.num_experts*3)}
        return qml.qnn.KerasLayer(president_tom, weight_shapes, output_dim=self.num_experts)
        
       
            
tf.get_logger().setLevel('ERROR')

In [ ]:
# Ising Version. my_model_ising
class QuantumLegislator(tfk.Model):
    def __init__(self, kernels=[5,3,3,5], conv_size=6, num_experts = 4, activation='sigmoid', pres_step_size=.85,batch_size=8):
        super(QuantumLegislator, self).__init__()
        self.num_experts = num_experts
        self.cabinet = [self.create_cabinet_member(kernels=kernels, conv_size=conv_size, activation = activation)] * num_experts
        #self.sample_dev = qml.device('default.qubit', wires=num_experts,shots=1)
        self.state_dev = qml.device('default.qubit', wires=num_experts)
        self.pres_opt = qml.optimize.AdamOptimizer(pres_step_size)
        #self.qvc_weights = np.random.rand(self.num_experts*3, requires_grad=True).reshape(self.num_experts,3)
        self.batch_size= batch_size
        self.pres = self.pres_layer()
        self.accuracy = tf.keras.metrics.CategoricalAccuracy()

    def train_step(self, data):
        x, e, y = data
        #print(type(y))
        #y_gt = np.array([y] * self.num_experts).reshape((self.num_experts,self.batch_size,10)) # num_experts x 10
        #print(y_gt.shape)
        with tf.GradientTape() as tape:
            tom_vetos = self.pres(inputs=e, training=True)[...,1]
            
            y_pred = np.array([cab(x, training = True) for cab in self.cabinet]) # num_experts x batch_size x 10
            cab_loss = [self.compiled_loss(y, cab(x, training = True)) for cab in self.cabinet]

            ## BRYAN LOSS
            tom_sum = tf.reduce_sum(tom_vetos)
            cab_sum = tf.reduce_sum(cab_loss)
            tom_val = tf.cast((tom_vetos - tom_sum), dtype = tf.float32)

            cab_val = tf.reshape(tf.cast((cab_loss-cab_sum), dtype = tf.float32),(self.num_experts))
            expert_losses = tom_val * cab_val
            
            final_loss = tf.reduce_sum(expert_losses)
            weights = self.trainable_variables   # num_experts 
            classic_gradients = tape.gradient(final_loss, weights)
            self.optimizer.apply_gradients(zip(classic_gradients, weights))
            self.accuracy.update_state(y, y_pred)

        return {'Full Loss':final_loss, 'Accuracy': self.accuracy.result()}
    
    def test_step(self, data):
        x, e, y = data
        
        tom_vetos = self.pres(inputs=e, training=True)[...,1].reshape(self.batch_size * self.num_experts) # 128,
        vetos = np.array([0 if np.random.rand() > x else 1 for x in tom_vetos]).reshape((1, self.batch_size, self.num_experts )).T # 4X32
        
        y_pred_ = np.array([cab(x, training = True) for cab in self.cabinet]) * vetos # num_experts
        y_pred = np.sum(y_pred_ ,axis=-1) # 32 X 10
        
        cab_loss = self.compiled_loss(y, y_pred)
        self.accuracy.update_state(y, y_pred)

        #return {'Validation Loss':cab_loss, 'Validation Accuracy': self.accuracy.result()}
        return {'Validation Accuracy': self.accuracy.result()}

    def create_cabinet_member(self, kernels=[5,3,3,5], conv_size=6, activation='sigmoid'):
        datum = tfl.Input(shape=(32,32,3),name='x')
        #init = tfk.initializers.Orthogonal()
        activation = activation

        c = tfpl.Convolution2DReparameterization(filters=conv_size, kernel_size=kernels[0], padding='valid', data_format='channels_last')(datum)
        x = tfpl.Convolution2DReparameterization(filters=conv_size, kernel_size=kernels[1], padding='valid', data_format='channels_last')(datum)
        x = tfl.Activation(activation)(x)
        x = tfpl.Convolution2DReparameterization(filters=conv_size, kernel_size=kernels[2], padding='valid', data_format='channels_last')(x)
        x = tfl.Add()([x, c])
        x = tfl.Activation(activation)(x)
        x = tfpl.Convolution2DReparameterization(filters=3, kernel_size=kernels[3], padding='valid', data_format='channels_last')(x)
        x = tfl.Activation(activation)(x)
        x = tfl.Flatten()(x)
        x = tfpl.DenseReparameterization(256, activation)(x)
        x = tfpl.DenseReparameterization(10, 'softmax')(x)

        return tfk.models.Model(inputs=datum, outputs=x)
    
    def pres_layer(self):
        #@qml.batch_input(argnum=1)
        @qml.qnode(self.state_dev, diff_method="backprop", interface="tf")
        def president_tom(inputs, weights):
            # x Shape = num_experts * 3
            # Weights Shape = num_experts * 2 * 3
            marker = 0
            for i in range(self.num_experts):
                qml.Hadamard(i)
                qml.Rot(inputs[..., 3*i], inputs[..., 3*i+1], inputs[..., 3*i+2], i)
                
            qml.IsingXY(weights[marker + 0],(self.num_experts-1, 0)) #entangling ring
            for i in range(1, self.num_experts):
                qml.IsingXY(weights[marker + i],(i-1, i))
                
            qml.CZ((0,2))    
            qml.CZ((1,3))    
            marker += self.num_experts
            qml.IsingZZ(weights[marker + 0],(self.num_experts-1, 0)) #entangling ring
            for i in range(1, self.num_experts):
                qml.IsingZZ(weights[marker + i],(i-1, i))
                
            qml.CZ((0,2))    
            qml.CZ((1,3))  
            
            marker += self.num_experts
            qml.IsingXX(weights[marker + 0],(self.num_experts-1, 0)) #entangling ring

## Training

In [84]:
#all_records = ['fma_small.tfrecords','openmic_imagedata.tfrecords','mnist_records.tfrecords','spoken_digit_records.tfrecords','IRMAS_records.tfrecords','fashion_mnist_records.tfrecords','cifar_records.tfrecords','svhn_records.tfrecords']
all_records  =['ICA_imagedata.tfrecords']
current_dataset = all_records[ : ]
data_size = 328318
batch_size = 32
t_t_split = .25
take_data = int((1 - t_t_split) * data_size)
epochs = 30
steps_per_epoch = int(take_data/batch_size/epochs)
validation_steps = int(epochs * t_t_split)
print(data_size, take_data)
#my_model = create_cabinet_member()
my_model = QuantumLegislator(num_experts = 4,batch_size=batch_size)
#data = tf.data.TFRecordDataset("fashion_mnist_records.tfrecords").map(decode_fn)
train_data = tf.data.TFRecordDataset(current_dataset).map(decode_fn).map(lambda x: (x['x'], x['e'], tf.one_hot(tf.cast(x['y'], tf.int64), 10))).take(take_data).shuffle(512).batch(batch_size)
test_data = tf.data.TFRecordDataset(current_dataset).map(decode_fn).map(lambda x: (x['x'], x['e'], tf.one_hot(tf.cast(x['y'], tf.int64), 10))).skip(take_data).shuffle(512).batch(batch_size)
#my_model.compile(tf.keras.optimizers.Adam(), 'mse')
my_model.compile(tf.keras.optimizers.Adamax(learning_rate=0.01), loss=tf.keras.losses.CategoricalCrossentropy())
my_model.fit(train_data, batch_size=batch_size,epochs=epochs,steps_per_epoch=steps_per_epoch, validation_data=test_data,validation_steps=validation_steps )

328318 246238
Epoch 1/30
149/256 [================>.............] - ETA: 7:47 - Full Loss: 53222.5840 - Accuracy: 0.1753

KeyboardInterrupt: 

In [14]:
checkpoint_path = "./my_model/"

In [15]:
my_model.save_weights(checkpoint_path.format(epoch=0))

## Testing

In [98]:
# Load model from saved checkpoint
my_model = QuantumLegislator(num_experts = 4, batch_size = batch_size)
my_model.load_weights(checkpoint_path)

In [ ]:
all_records  =['ICA_imagedata.tfrecords']
current_dataset = all_records[ : ]
data_size = 328318
batch_size = 32
t_t_split = .25
take_data = int((1 - t_t_split) * data_size)

test_data = tf.data.TFRecordDataset(current_dataset).map(decode_fn).map(lambda x: (x['x'], x['e'], tf.one_hot(tf.cast(x['y'], tf.int64), 10))).skip(take_data).batch(batch_size).shuffle(512)
my_model.compile(tf.keras.optimizers.Adamax(learning_rate=0.01), tf.keras.losses.CategoricalCrossentropy())
my_model.evaluate(test_data)

    748/Unknown - 1778s 2s/step - Validation Loss: nan - Validation Accuracy: 0.1098